In [16]:
# Importation of modules
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage import filters
from time import perf_counter
import os
from cropping import masking
from pprint import pprint
import time
%matplotlib inline

In [79]:
# Create a dictionary location:value --> {'270-01-01': 's_2-s_7-s_0-s_pt',...}

def get_transcription(transcription_file):
    with open(transcription_file, 'r') as f:
        names = {}
        for line in f.readlines():
            names[line.split(" ")[0]] = line.split(" ")[1].strip()
    return names

def get_list_of_text(file):
    with open(file, 'r') as f:
        nameOfText = []
        for line in f.readlines():
            nameOfText.append(line.strip())
    return nameOfText

transciptionOfAllWords = get_transcription("./../ground-truth/transcription.txt")
trainingFiles = get_list_of_text("./../task/train.txt")
validationFiles = get_list_of_text("./../task/valid.txt")
print(">> Dictionary and sets created")


>> Dictionary and sets created


In [20]:
# Just do that one time to generate the .pkl files -> If files are already available skip this cell
def cropPictures():
    allPictures = os.listdir("./../images")

    if not "pkls" in os.listdir("."):
        os.makedirs("./pkls")

    for picture in allPictures:
    
        print(">> Cropping picture", picture[:-4], " ...")
        start_time = time.perf_counter()
        numberOfPicture = picture[:-4]

        # Get all split words
        trimmed = masking(int(numberOfPicture))
    
        # Serialize the objects with pickle --> creation of all .pkl files
        with open(os.path.join("./pkls", numberOfPicture), 'wb') as f:
            pickle.dump(trimmed, f, pickle.HIGHEST_PROTOCOL)
        print(" Picture %s cropped in: %ss" % (picture[:-4], int(time.perf_counter()-start_time)))
        
cropPictures()

>> Cropping picture 270  ...
 Picture 270 cropped in: 198s
>> Cropping picture 271  ...
 Picture 271 cropped in: 279s
>> Cropping picture 272  ...
 Picture 272 cropped in: 235s
>> Cropping picture 273  ...
 Picture 273 cropped in: 252s
>> Cropping picture 274  ...
 Picture 274 cropped in: 316s
>> Cropping picture 275  ...
 Picture 275 cropped in: 313s
>> Cropping picture 276  ...
 Picture 276 cropped in: 220s
>> Cropping picture 277  ...
 Picture 277 cropped in: 270s
>> Cropping picture 278  ...
 Picture 278 cropped in: 177s
>> Cropping picture 279  ...
 Picture 279 cropped in: 292s
>> Cropping picture 300  ...
 Picture 300 cropped in: 183s
>> Cropping picture 301  ...
 Picture 301 cropped in: 330s
>> Cropping picture 302  ...
 Picture 302 cropped in: 336s
>> Cropping picture 303  ...
 Picture 303 cropped in: 329s
>> Cropping picture 304  ...
 Picture 304 cropped in: 219s


In [49]:
def binarizeAndStoreInDict(typeOfSet):
    allPkls = os.listdir("./pkls")

    #if not "cropped" in os.listdir("."):
        #os.makedirs("./cropped")

    # Dictionary containing the identifier and the values of all pixels of each word
    allWordsAsNumpyMatrix={}
    height=0
    width=0
    
    # Get maximum width and maximum height of all words in order to create a picture of same size 
    # for all words of all pages
    for text in allPkls:
        with open('./pkls/'+text,'rb') as f:
            allWords = pickle.load(f)
            maxHeightPage = max([x.shape[0] for x in allWords.values()])
            maxWidthPage = max([x.shape[1] for x in allWords.values()])
            if maxHeightPage>height:
                height= maxHeightPage
            if maxWidthPage > width:
                width= maxWidthPage

    for text in allPkls:
        if text not in typeOfSet:
            continue
        with open('./pkls/'+text,'rb') as f:
            allWords = pickle.load(f)
            
            for wordpkl in allWords:
                word = allWords[wordpkl].filled(255)
            
                # Only keep the dark parts
                tresh = filters.threshold_otsu(word)
                foo = word<tresh
            
                # Set every "word" as a matrix of same size.
                target = np.zeros((height,width))
                target[:foo.shape[0],:foo.shape[1]]=foo
            
                # return a dictionary '304-01-01':[[0,0,0,0,0,0,1,...,0,..], [], [0,1,0,0,..], []..]
                allWordsAsNumpyMatrix[wordpkl]=target
                
    return allWordsAsNumpyMatrix
allWords_training = binarizeAndStoreInDict(trainingFiles)
allWords_validation = binarizeAndStoreInDict(validationFiles)

In [71]:
# JUST A TEST TO SEE IF ARRAYS OF FEATURES ARE WELL CREATED
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# Transform each picture of word into an array of features and store it in a dictionary identifier: allfeatures
# --> {'304-01-01': [feature1, feature2, feature3, feature4, feature5]}
from features import *
allWords_training_features = calculate_features_of_all_samples(allWords_training)





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
#Just a test to compare the distance of an image with itself (should equals 0) and an image with another one
from dtw import *

score1 = dtw(allWords_training_features['270-09-06'],allWords_training_features['270-05-07'])
score2 = dtw(allWords_training_features['270-09-06'],allWords_training_features['270-09-06'])
print(score1)
print(score2)

13871.362575372024
0.0


In [ ]:
## IS TOO SLOW FOR NOW... NOT TESTED
import operator

def kNN_DTW(allWords_training, allWords_test, k, transcriptionOfAllWords):
# Returns the k nearest neighbors through DTW distance algorithm
    allWords_training_features = calculate_features_of_all_samples(allWords_training)
    allWords_test_features = calculate_features_of_all_samples(allWords_test)
    
    for test_image in allWords_test_features.keys():
        # compute distance between each training instance and the test instance
        # and store tuple of result and corresponding training instance
        distances = []
        for training_image in allWords_training_features.keys():
            dist = dtw(allWords_test_features[test_image], allWords_training_features[training_image])
            distances.append((transciptionOfAllWords[training_image], dist, training_image))
        # sort list of tuples of distances in ascending order 
        # regarding the second item of the tuples, i.e. the distances
        distances.sort(key=operator.itemgetter(1))
        kNNforMajority = []
        locationOfWords = []
        for x in range(k):
            kNN.append(distances[x][0])
            locationOfWords.append(distances[x][2])
        # return the k first training instances (the k most similar)
        return majority(kNN), locationOfWords


def majority(kNN):
# Returns the label occuring the most among the k nearest neighbors
    # classVotes is a key:value list
    classVotes = {}
    for x in range(len(kNN)):
        # extract the label of training instance 
        response = kNN[x][0]
        # if the label appears in list of classVotes 
        if response in classVotes:
        # add vote at to label key
            classVotes[response] += 1  
        else:
            classVotes[response] = 1
    # sort the classVotes according to the number of votes in a descending order
    sortedVotes = sorted(classVotes.iteritems(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

# test with one word
label, locations = kNN_DTW(allWords_training, {'304-32-03':allWords_validation['304-32-03']},10, transciptionOfAllWords)
print("The words that are found are", label)
print("They were found at ", locations)
